In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pydot --quiet
!pip install tensorflow==2.15.0 --quiet
!pip install transformers --quiet
!pip install sentencepiece --quiet
!pip install datasets --quiet
!pip install accelerate --quiet
!pip install transformers[torch] --quiet
!pip install evaluate --quiet
!pip install rouge_score --quiet
!pip install jiwer --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
import transformers
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd
import accelerate
import torch
from torch.utils.data import Dataset
from evaluate import load
from huggingface_hub import notebook_login
pd.set_option('display.max_colwidth', 0)

In [ ]:
notebook_login()

In [ ]:
df = load_dataset("billsum")

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

## 2. Summary Stats

In [ ]:
bill_lengths = {"train": [], "test": [], "ca_test":[]}
bill_stats = {"train": {}, "test": {}, "ca_test":{}}

for split in df.keys():
	for i in range(len(df[split])):
		bill_lengths[split].append(len(df[split][i]["text"]))

	bill_stats[split]["mean"] = sum(bill_lengths[split]) / len(bill_lengths[split])
	bill_stats[split]["max"] = max(bill_lengths[split])
	bill_stats[split]["min"] = min(bill_lengths[split])
	bill_stats[split]["std"] = np.std(bill_lengths[split])
	bill_stats[split]["median"] = np.median(bill_lengths[split])

print(bill_stats['train'])
print(bill_stats['test'])
print(bill_stats['ca_test'])

{'mean': 10272.542931025384, 'max': 19998, 'min': 5001, 'std': 4120.465966900558, 'median': 9283.0}
{'mean': 10268.095442031203, 'max': 19998, 'min': 5004, 'std': 4149.99403259188, 'median': 9316.0}
{'mean': 9729.51818916734, 'max': 19998, 'min': 5006, 'std': 3920.693393960155, 'median': 8555.0}


In [ ]:
count = 0

for input in df['train']:
  word_count = len(input['text'].split())

  if word_count < 1000:
    count += 1

print(count)

7099


In [ ]:
count = 0

for input in df['test']:
  word_count = len(input['text'].split())

  if word_count < 1000:
    count += 1

print(count)

1248


In [ ]:


limit = 1000

def word_limit(input):
  word_count = len(input['text'].split())
  return word_count <= limit

df = df.filter(word_limit)

df



DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 7113
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1249
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 273
    })
})

In [ ]:
# ====== ENVIRONMENT ======
DEV = False
EXPLORE = False
RANDOM_SEED = 42

# ====== DATA =======
INPUT_MAX_LENGTH = 1024
LABEL_MAX_LENGTH = 128

# ====== MODEL ======
MODEL_NAME = "BART_1st_STAGE_SUMMARIZER_v3"
HUGGINGFACE_DIR = "jgibb/" + MODEL_NAME
CHECKPOINT = "facebook/bart-large"
PATH = '/content/drive/MyDrive/colab-notebooks/w266_project/'
PEFT = False

# ====== OPTIIZER =======
OPTIMIZER = "adamw_bnb_8bit"
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

# ====== TRAINING ======
EVAL_STRATEGY = "steps"
BATCH_SIZE = 6
EPOCHS = 3
SAVE_STEPS = 100 if DEV else 1000
EVAL_STEPS = 100 if DEV else 250
LOGGING_STEPS = 100 if DEV else 500
METRIC_WER = load("wer")
METRIC_ROUGE = load("rouge")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
def model_init():
  seed = 42
  torch.manual_seed(seed)
  return AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)
#model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_data_baseline(df):
  input = [doc for doc in df["text"]]
  #tokenize full text
  model_inputs = tokenizer(input, max_length=INPUT_MAX_LENGTH, return_tensors = "pt", padding=True, truncation=True)
  #tokenize the summaries
  summary_input = [doc for doc in df["summary"]]
  targets = tokenizer(text_target = summary_input, max_length=LABEL_MAX_LENGTH, padding= True, truncation=True)
  #set labels
  model_inputs['labels'] = targets['input_ids']
  return model_inputs

In [ ]:
tokenized_data = df.map(preprocess_data_baseline, batched = True)

Map:   0%|          | 0/7113 [00:00<?, ? examples/s]

Map:   0%|          | 0/1249 [00:00<?, ? examples/s]

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=CHECKPOINT)

In [ ]:
def compute_metrics(pred):

    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    result_wer = METRIC_WER.compute(predictions=pred_str, references=label_str)
    result_rouge = METRIC_ROUGE.compute(predictions=pred_str, references=label_str, use_stemmer=True, use_aggregator=True)
    result_rouge = {key: value for key, value in result_rouge.items()}

    results = {key: value for key, value in result_rouge.items()}
    results["WER"] = result_wer
    return {k: round(v, 4) for k,v in results.items()}

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir = HUGGINGFACE_DIR,
    evaluation_strategy=EVAL_STRATEGY,
    save_strategy=EVAL_STRATEGY,
    save_steps=SAVE_STEPS,
    eval_steps=EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    #eval_accumulation_steps=16,
    load_best_model_at_end=True,
    num_train_epochs=EPOCHS,
    # predict_with_generate=True,
    fp16=True,
    report_to=["tensorboard"],
    push_to_hub=True,
    )

trainer = Seq2SeqTrainer(
    #model = model,
    model_init=model_init,
    args = args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Wer
250,No log,1.364733,0.719500,0.483600,0.658900,0.663900,0.417400
500,1.954000,1.250545,0.730100,0.498500,0.669800,0.674400,0.406000
750,1.954000,1.209355,0.734100,0.505100,0.676400,0.679900,0.397300
1000,1.363500,1.177090,0.738700,0.514400,0.683700,0.688000,0.390200
1250,1.363500,1.186158,0.742200,0.517700,0.685300,0.690700,0.389200
1500,1.224600,1.151352,0.741600,0.518200,0.686000,0.690500,0.386900
1750,1.224600,1.141467,0.744800,0.521900,0.688400,0.693000,0.383700
2000,1.159200,1.123502,0.745900,0.525200,0.691000,0.695300,0.380000
2250,1.159200,1.121023,0.748100,0.528400,0.692100,0.697000,0.377800
2500,1.108400,1.107392,0.748700,0.529500,0.693900,0.698700,0.376000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=3558, training_loss=1.2646169882265619, metrics={'train_runtime': 4015.6274, 'train_samples_per_second': 5.314, 'train_steps_per_second': 0.886, 'total_flos': 4.624384510446797e+16, 'train_loss': 1.2646169882265619, 'epoch': 3.0})

In [ ]:
trainer.create_model_card(
    language='english',
    model_name= MODEL_NAME,
    tasks='summarization',
    tags='summarization',
    dataset='BILLSUM',
    dataset_args=f"Max token input: {INPUT_MAX_LENGTH} | {LABEL_MAX_LENGTH}"
)

commit_msg = f"training completed[dev]: {INPUT_MAX_LENGTH} {LABEL_MAX_LENGTH}"
trainer.push_to_hub(commit_message=commit_msg)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


events.out.tfevents.1711083704.9ddc280fa2d1.2304.0:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jgibb/BART_1st_STAGE_SUMMARIZER_v3/commit/e59952cd8ce7992dc6acefa163afb2cbe969ca8e', commit_message='training completed[dev]: 1024 128', commit_description='', oid='e59952cd8ce7992dc6acefa163afb2cbe969ca8e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
predict_results = trainer.predict(tokenized_data["ca_test"])
trainer.save_metrics(f"BART_SUMMARIZER_{INPUT_MAX_LENGTH}", predict_results.metrics)

In [ ]:
import pprint
pprint.pprint(predict_results.metrics)

{'test_WER': 0.5373,
 'test_loss': 2.1799163818359375,
 'test_rouge1': 0.6544,
 'test_rouge2': 0.3638,
 'test_rougeL': 0.5555,
 'test_rougeLsum': 0.5626,
 'test_runtime': 14.429,
 'test_samples_per_second': 18.92,
 'test_steps_per_second': 3.188}
